In [14]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, Lipinski, Fragments
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score, classification_report

In [2]:
training_data_path = 'training_smiles.csv'
test_data_path = 'test_smiles.csv'

training_data = pd.read_csv(training_data_path, dtype = {'ACTIVE': int})
test_data = pd.read_csv(test_data_path)

In [3]:
def extract_fingerprints(smiles):
    mol = Chem.MolFromSmiles(smiles)

    features = {}

    # Try nBits 2048, 1024, 512, 256
    # Morgan Fingerprint
    morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=512)
    for i in range(512):
        features[f'fp_{i}'] = morgan_fp[i]

    return features


    
training_features_df = training_data['SMILES'].apply(extract_fingerprints)

training_features_df = training_features_df.apply(pd.Series)

training_data_fingerprint = training_data.join(training_features_df)

training_data_fingerprint.to_csv('training_data_fingerprint.csv', index=False)


[11:00:31] WARNING: not removing hydrogen atom without neighbors
C:\Users\hyde1\AppData\Local\Temp\ipykernel_3452\2814264457.py:18: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  training_features_df = training_features_df.apply(pd.Series)


In [4]:

def extract_features(smiles):

    mol = Chem.MolFromSmiles(smiles)

    features = {}

    # Basic Properties
    features['num_atoms'] = mol.GetNumAtoms()
    features['num_bonds'] = mol.GetNumBonds()
    features['num_rings'] = mol.GetRingInfo().NumRings()

    # Molecular Descriptors
    for desc_name, desc_func in Descriptors.descList:
        features[desc_name] = desc_func(mol)

    # Lipinski Descriptors
    features['num_rotatable_bonds'] = Lipinski.NumRotatableBonds(mol)
    features['num_aromatic_rings'] = Lipinski.NumAromaticRings(mol)
    features['num_heteroatoms'] = Lipinski.NumHeteroatoms(mol)
    features['num_heavy_atoms'] = Lipinski.HeavyAtomCount(mol)
    features['num_h_donors'] = Lipinski.NumHDonors(mol)
    features['num_h_acceptors'] = Lipinski.NumHAcceptors(mol)
    features['num_aliphatic_rings'] = Lipinski.NumAliphaticRings(mol)
    features['num_saturated_rings'] = Lipinski.NumSaturatedRings(mol)
    features['num_aromatic_heterocycles'] = Lipinski.NumAromaticHeterocycles(mol)
    features['num_aromatic_carbocycles'] = Lipinski.NumAromaticCarbocycles(mol)
    features['num_aliphatic_heterocycles'] = Lipinski.NumAliphaticHeterocycles(mol)
    features['num_aliphatic_carbocycles'] = Lipinski.NumAliphaticCarbocycles(mol)

    # Fragment Descriptors
    for frag_func in dir(Fragments):
        if frag_func.startswith('fr_'):
            features[frag_func] = getattr(Fragments, frag_func)(mol)


    return features

training_features_df = training_data['SMILES'].apply(extract_features)

training_features_df = training_features_df.apply(pd.Series)

training_data_features = training_data.join(training_features_df)

training_data_features.to_csv('training_data_features.csv', index=False)



[11:27:42] WARNING: not removing hydrogen atom without neighbors
[11:27:42] WARNING: not removing hydrogen atom without neighbors
[11:27:42] WARNING: not removing hydrogen atom without neighbors
C:\Users\hyde1\AppData\Local\Temp\ipykernel_3452\2222261554.py:40: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  training_features_df = training_features_df.apply(pd.Series)


In [13]:
# Load the datasets
fingerprint_df = pd.read_csv('training_data_fingerprint.csv')
other_features_df = pd.read_csv('training_data_features.csv')

# Combine datasets
combined_df = pd.concat([fingerprint_df, other_features_df], axis=1)

columns_to_impute = [
    'MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge',
    'MinAbsPartialCharge', 'BCUT2D_MWHI', 'BCUT2D_MWLOW',
    'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW',
    'BCUT2D_MRHI', 'BCUT2D_MRLOW'
]

# Perform mean imputation
for column in columns_to_impute:
    combined_df[column].fillna(combined_df[column].mean(), inplace=True)

print(combined_df[columns_to_impute].isnull().sum())

MaxPartialCharge       0
MinPartialCharge       0
MaxAbsPartialCharge    0
MinAbsPartialCharge    0
BCUT2D_MWHI            0
BCUT2D_MWLOW           0
BCUT2D_CHGHI           0
BCUT2D_CHGLO           0
BCUT2D_LOGPHI          0
BCUT2D_LOGPLOW         0
BCUT2D_MRHI            0
BCUT2D_MRLOW           0
dtype: int64


In [4]:
combined_df = combined_df.drop(['INDEX', 'SMILES','ACTIVE'], axis=1)

In [4]:
combined_df.head()

,fp_0,fp_1,fp_2,fp_3,fp_4,fp_5,fp_6,fp_7,fp_8,fp_9,...,num_heteroatoms,num_heavy_atoms,num_h_donors,num_h_acceptors,num_aliphatic_rings,num_saturated_rings,num_aromatic_heterocycles,num_aromatic_carbocycles,num_aliphatic_heterocycles,num_aliphatic_carbocycles
0,0,0,0,0,0,0,0,0,0,0,...,3.0,16.0,1.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,7.0,26.0,1.0,6.0,0.0,0.0,2.0,1.0,0.0,0.0
2,0,0,0,0,1,0,0,0,0,0,...,8.0,26.0,1.0,4.0,3.0,2.0,0.0,0.0,2.0,1.0
3,0,0,0,0,0,0,1,0,0,0,...,8.0,27.0,2.0,5.0,0.0,0.0,1.0,2.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,4.0,20.0,1.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0


In [9]:
X = combined_df
y = other_features_df['ACTIVE']


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

rf_clf.fit(X_train, y_train)

importances = rf_clf.feature_importances_

feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
})

feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

feature_importance_df.to_csv('feature_importances.csv', index=False)

print(feature_importance_df.head())


               Feature  Importance
540       BCUT2D_MRLOW    0.010275
518     MinEStateIndex    0.009062
517  MinAbsEStateIndex    0.008667
614        VSA_EState4    0.008464
638            MolLogP    0.008230


In [13]:
feature_importances = pd.read_csv('feature_importances.csv')

threshold = 0.0005

selected_features = feature_importances[feature_importances['Importance'] > threshold]['Feature']

print(selected_features)

0           BCUT2D_MRLOW
1         MinEStateIndex
2      MinAbsEStateIndex
3            VSA_EState4
4                MolLogP
             ...        
515               fp_485
516               fp_394
517               fp_422
518               fp_469
519               fp_493
Name: Feature, Length: 520, dtype: object


In [15]:
# X = combined_df[selected_features]
# y = other_features_df['ACTIVE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

rf_clf.fit(X_train, y_train)

y_probs = rf_clf.predict_proba(X_test)[:, 1]

roc_auc = roc_auc_score(y_test, y_probs)
print(f"ROC AUC Score: {roc_auc}")


ROC AUC Score: 0.7613687115066163


In [16]:
#X = combined_df[selected_features]
#y = other_features_df['ACTIVE']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

rf_clf.fit(X_train_smote, y_train_smote)

y_pred = rf_clf.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC AUC Score: {roc_auc}")

print(classification_report(y_test, y_pred))


ROC AUC Score: 0.5147121548435469
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     45463
           1       0.60      0.03      0.06       506

    accuracy                           0.99     45969
   macro avg       0.79      0.51      0.53     45969
weighted avg       0.99      0.99      0.98     45969

